In [2]:
import os

import pandas as pd
from pandas import DataFrame
import numpy as np
from scipy import stats

from Constants import Constants as const

In [2]:
# Step 1: Load the data and create event time variable
file_path = os.path.join(const.RESULT_PATH, '20250113_stock_act_data_v5.dta')
df = pd.read_stata(file_path).copy()

C:\Users\wangy\AppData\Local\Temp\ipykernel_32088\1387148850.py:3: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(file_path).copy()


In [3]:
# Columns to be included in the analysis
columns = [
    "log_frequency_w", "log_frequency_ann_w", "log_frequency_qtr_w", "log_frequency_eps_w", "log_frequency_noneps_w", "log_frequency_day_w", 
    "env_info_w", "social_info_w", "width_w", "cpie_gpin", "cpie_owr", "SYNCHRONICITY_MONTH", "SYNCHRONICITY_MKT_MONTH", 
    "SYNCHRONICITY_IND_MONTH", "numest_last", "FCSTERROR_last", "DISPERSION_last", 
    "log_market_value_w", "lev_w", "BM_w", "ROA_w", "LOSS", "EarnVol_w", "ret_w", "turnover_w", "StkVol_w"
]


In [4]:
# Split the data into two parts based on fiscal_year
before_2012_df = df[df['fiscal_year'] < 2012]
after_2012_df = df[df['fiscal_year'] >= 2012]

In [5]:
# Function to perform comparison between treatment and control groups
def perform_comparison(df, columns):
    treatment_group = df[df['MajorGovCustomer'] == 1]
    control_group = df[df['MajorGovCustomer'] == 0]

    results = []

    for col in columns:
        treat_mean = treatment_group[col].mean()
        treat_median = treatment_group[col].median()
        treat_num = treatment_group[col].count()

        control_mean = control_group[col].mean()
        control_median = control_group[col].median()
        control_num = control_group[col].count()

        diff_mean = treat_mean - control_mean
        t_stat, p_value = stats.ttest_ind(treatment_group[col].dropna(), control_group[col].dropna(), equal_var=False)

        # Determine significance level
        if p_value < 0.01:
            significance = '***'
        elif p_value < 0.05:
            significance = '**'
        elif p_value < 0.1:
            significance = '*'
        else:
            significance = ''

        # Round values to three significant digits
        treat_mean = round(treat_mean, 3)
        treat_median = round(treat_median, 3)
        control_mean = round(control_mean, 3)
        control_median = round(control_median, 3)
        diff_mean = round(diff_mean, 3)
        t_stat = f"({round(t_stat, 3)})"

        results.append([treat_mean, treat_median, treat_num, control_mean, control_median, control_num, diff_mean, t_stat, significance])

    # Create a DataFrame to display the results
    results_df = pd.DataFrame(results, columns=[
        "Treatment Mean", "Treatment Median", "Treatment N", 
        "Control Mean", "Control Median", "Control N", 
        "Difference in Mean", "t-stat", "Significance"
    ], index=columns)
    return results_df

In [6]:
# Perform comparison for data before 2012
results_before_2012 = perform_comparison(before_2012_df, columns)
print("Results for data before 2012:")
print(results_before_2012)


Results for data before 2012:
                         Treatment Mean  Treatment Median  Treatment N  \
log_frequency_w                   0.752             0.000          933   
log_frequency_ann_w               0.684             0.000          933   
log_frequency_qtr_w               0.225             0.000          933   
log_frequency_eps_w               0.351             0.000          933   
log_frequency_noneps_w            0.656             0.000          933   
log_frequency_day_w               0.545             0.000          933   
env_info_w                        3.815             0.000          933   
social_info_w                    11.170            12.023          933   
width_w                           0.689             0.800          209   
cpie_gpin                         0.426             0.474          517   
cpie_owr                          0.427             0.477          517   
SYNCHRONICITY_MONTH               1.583             1.520          612   
SYNCHRON

In [7]:
results_before_2012.to_excel(os.path.join(const.REGRESSION_RESULT_PATH, '20250126', 'T2PA TvC before 2012.xlsx'))

In [8]:
# Perform comparison for data from 2012 onwards
results_after_2012 = perform_comparison(after_2012_df, columns)
print("Results for data from 2012 onwards:")
print(results_after_2012)

Results for data from 2012 onwards:
                         Treatment Mean  Treatment Median  Treatment N  \
log_frequency_w                   0.750             0.000          921   
log_frequency_ann_w               0.676             0.000          921   
log_frequency_qtr_w               0.229             0.000          921   
log_frequency_eps_w               0.363             0.000          921   
log_frequency_noneps_w            0.651             0.000          921   
log_frequency_day_w               0.545             0.000          921   
env_info_w                        3.180             0.000          921   
social_info_w                    11.640            12.616          921   
width_w                           0.732             0.800          139   
cpie_gpin                         0.453             0.486          657   
cpie_owr                          0.417             0.431          657   
SYNCHRONICITY_MONTH               1.313             1.234          624   
SY

In [9]:
results_after_2012.to_excel(os.path.join(const.REGRESSION_RESULT_PATH, '20250126', 'T2PB TvC after 2012.xlsx'))


# Get stepwise observations number

In [2]:
ctat_df = pd.read_csv(os.path.join(const.COMPUSTAT_PATH, '1950_2022_ctat_all_data.zip'),
                                     usecols=['fyear', const.GVKEY, 'bkvlps', 'prcc_f', 'csho', 'mkvalt'],
                                     dtype={const.GVKEY: str}).rename(columns={'fyear': const.YEAR})
ctat_df = ctat_df.loc[ctat_df[const.YEAR].apply(lambda x: 2007 < x < 2016)]
ctat_df.shape

(99932, 6)

In [3]:
ctat_df.drop_duplicates(subset=['gvkey', const.YEAR]).shape

(90226, 6)

# Calculate the cost of capital observations

In [2]:
lee_df: DataFrame = pd.read_csv(
            os.path.join(const.DATABASE_PATH, 'Cost of Capital', f'erp_public_annual_240107.zip')).drop(['permno'], axis=1)

In [5]:
lee_df[['CCC', 'GLS_mech', 'OJM_mech', 'CAT_mech', 'PEG_mech']].describe().T

,count,mean,std,min,25%,50%,75%,max
CCC,2088311.0,0.090159,0.124445,-0.538879,0.007939,0.065900,0.145795,0.999968
GLS_mech,2068379.0,0.112307,0.072132,-0.683782,0.070185,0.099579,0.139144,0.999780
OJM_mech,1768009.0,0.111107,0.137101,-0.990699,0.039521,0.071381,0.130939,0.999980
CAT_mech,2054667.0,-0.006180,0.230304,-0.920344,-0.156598,-0.019888,0.129285,0.999998
PEG_mech,2080521.0,0.140521,0.142460,0.000000,0.046016,0.106245,0.190371,0.999995


In [9]:
lee_df.loc[lee_df['CCC'].notnull() & lee_df['GLS_mech'].isnull(), ['CCC', 'GLS_mech', 'OJM_mech', 'CAT_mech', 'PEG_mech']]

,CCC,GLS_mech,OJM_mech,CAT_mech,PEG_mech
641,0.587013,NaN,NaN,NaN,0.587013
642,0.502280,NaN,NaN,NaN,0.502280
643,0.467239,NaN,NaN,NaN,0.467239
644,0.451498,NaN,NaN,NaN,0.451498
645,0.399652,NaN,NaN,NaN,0.399652
...,...,...,...,...,...
3481805,0.161550,NaN,NaN,NaN,0.161550
3481806,0.189709,NaN,NaN,NaN,0.189709
3481807,0.191881,NaN,NaN,NaN,0.191881
3481808,0.198166,NaN,NaN,NaN,0.198166


# Check missing car variables

In [3]:
data_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20250316_stock_act_idiosyn_v2.dta'))

In [4]:
data_df.loc[data_df['IDIOSYN_MONTH'].isnull()]

,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,ROA,...,QFM,T,propensity_score,is_psm,IDIOSYN_MONTH,IDIOSYN_MKT_MONTH,IDIOSYN_IND_MONTH,sigma,skewness,kurtosis
16,1076,2008,0.0,0.0,0.000000,0.0,0.0,7.264844,0.093100,0.069546,...,0.463002,0.0,0.145340,0,NaN,NaN,NaN,0.089141,0.285110,-0.541172
17,1076,2009,0.0,0.0,0.000000,0.0,0.0,7.316561,0.041654,0.085419,...,1.443899,0.0,0.140719,0,NaN,NaN,NaN,0.144905,0.495102,-0.584892
18,1076,2010,0.0,0.0,0.000000,0.0,0.0,7.398770,0.027822,0.078808,...,0.592875,0.0,0.138018,0,NaN,NaN,NaN,0.146767,1.033412,1.400927
19,1076,2011,0.0,0.0,0.000000,0.0,0.0,7.610395,0.088632,0.065566,...,0.405138,0.0,0.141469,0,NaN,NaN,NaN,0.091929,0.705460,0.029511
20,1076,2012,0.0,0.0,0.000000,0.0,1.0,7.669664,0.078066,0.095449,...,0.390994,0.0,0.142654,0,NaN,NaN,NaN,0.058585,0.036569,-1.033455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12069,293884,2015,1.0,12.0,2.564949,0.0,1.0,7.653997,0.443877,0.038789,...,0.337576,0.0,0.181435,0,NaN,NaN,NaN,0.077518,0.256709,-0.087653
12070,296753,2012,0.0,0.0,0.000000,0.0,1.0,6.752411,0.300591,0.141644,...,NaN,0.0,0.178608,0,NaN,NaN,NaN,0.143973,0.252652,-0.030403
12071,296753,2013,0.0,0.0,0.000000,0.0,1.0,6.819720,0.097789,0.207685,...,-0.102551,0.0,0.158537,0,NaN,NaN,NaN,0.126620,-0.188029,-1.545721
12072,296753,2014,0.0,0.0,0.000000,0.0,1.0,6.927963,0.348357,0.084109,...,0.246454,0.0,0.178111,0,NaN,NaN,NaN,0.089098,-0.512927,-0.727408
